In [1]:
import os
from glob import glob
import numpy as np
import random
import cv2
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import keras_cv

2023-11-22 17:55:36.377281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 17:55:37.003547: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
devices = tf.config.list_physical_devices()
print(devices)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2023-11-22 17:55:46.205230: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-22 17:55:46.235608: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
classifier = keras_cv.models.ImageClassifier.from_preset(
    "efficientnetv2_b0_imagenet_classifier"
)

In [ ]:
images_dir = os.path.abspath("../data/images/")

image_categories = os.listdir(images_dir)
print(image_categories)

In [ ]:

image_paths_dict = dict()
for category in image_categories:
    paths = glob(os.path.join(images_dir, category, "*"))
    image_paths_dict[category] = paths

In [ ]:
for category, paths in image_paths_dict.items():
    print(category)
    images = [np.array(keras.utils.load_img(path)) for path in paths[:4]]
    fig = plt.figure()
    grid = ImageGrid(fig, 111, nrows_ncols=(2, 2))

    for ax, im in zip(grid, images):
        ax.imshow(im)
        ax.axis("off")
    plt.show()

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    images_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=32,
)

valid_ds = keras.utils.image_dataset_from_directory(
    images_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(256, 256),
    batch_size=32,
)

In [ ]:
for images, labels in train_ds.take(1):
    print(labels)

In [ ]:
loader = keras.utils.image_dataset_from_directory(
    images_dir,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
train_ds.class_names

In [ ]:
for batch in loader:
    print(batch)
    break

In [ ]:
data = []
for i, paths in enumerate(image_paths_dict.values()):
    for path in paths:
        elem = i, path
        data.append(elem)
random.shuffle(data)

for elem in data[:5]:
    print(elem)

In [ ]:
def loader(image_paths_dict, batch_size):
    data = []
    for i, paths in enumerate(image_paths_dict.values()):
        for path in paths:
            elem = i, path
            data.append(elem)
    